In [ ]:
from main.compiling.compilers.AncillaPerCheckCompiler import AncillaPerCheckCompiler
from main.compiling.noise.models.CircuitLevelNoise import CircuitLevelNoise
from main.compiling.syndrome_extraction.extractors.ancilla_per_check.mixed.CxCyCzExtractor import CxCyCzExtractor
from main.codes.tic_tac_toe.HoneycombCode import HoneycombCode
from main.codes.tic_tac_toe.gauge_honeycomb_code import GaugeHoneycombCode
from main.codes.tic_tac_toe.FloquetColourCode import FloquetColourCode
from main.codes.tic_tac_toe.TicTacToeCode import TicTacToeCode
from main.utils.enums import State
from main.building_blocks.pauli import Pauli
from main.building_blocks.pauli.PauliLetter import PauliLetter
import stim
import sinter 
import matplotlib.pyplot as plt
import numpy as np
from beliefmatching import BeliefMatchingSinterDecoder
from main.utils.Colour import Blue, Green, Red


class HoneycombCodeZ(TicTacToeCode):
    """Adaptation of the Honeycomb code where the measurement order has been permuted
    """
    def __init__(self, distance: int):
        tic_tac_toe = [
                (Blue, PauliLetter('Z')),
                (Red, PauliLetter('X')),
                (Green, PauliLetter('Y'))]

        super().__init__(distance, tic_tac_toe)

# load the data.
# seperate running and loading.



# Help functions

In [7]:
def calculate_threshold(code_name, distances, observable_index, layer_factor,pers):
    tasks = []
    if code_name == "HoneycombCodeX":
        code = HoneycombCode
    elif code_name == "HoneycombCodeZ":
        code = HoneycombCodeZ
    elif code_name == "GaugeHoneycombCodeX":
        code = GaugeHoneycombCode
    elif code_name == "FloquetColourCodeX":
        code = FloquetColourCode
    

    for d in distances:
        for index, p in enumerate(pers):

            # generate circuit
            if code_name[-1] == 'X':
                circuit = get_X_stim_circuit(code(d), observable_index,layer_factor*d, p)
            elif code_name[-1] == 'Z':
                circuit = get_Z_stim_circuit(code(d), observable_index, layer_factor*d, p)
            tasks.append(sinter.Task(
                    circuit=circuit,
                    
                    #detector_error_model=circuit.detector_error_model(approximate_disjoint_errors=True, ignore_decomposition_failures=True),
                    json_metadata={
                        'code': code_name,
                        'distance': d,
                        'p': p,
                        'layers': layer_factor*d}))

    samples = sinter.collect(
        tasks=tasks,
        hint_num_tasks=len(tasks),
        num_workers=8,
        max_shots=5000,
        max_errors=500,
        decoders=['beliefmatching'],
        custom_decoders={'beliefmatching': BeliefMatchingSinterDecoder()},
        print_progress=True,
        save_resume_filepath=f'./resume/{code_name}_{observable_index}_{layer_factor}_belief_matching.json',)
    return(samples)


def get_X_stim_circuit(code,observable_index,layers,p):
    compiler = AncillaPerCheckCompiler(CircuitLevelNoise(p,p,p,p,p), syndrome_extractor=CxCyCzExtractor())
    data_qubits = code.data_qubits.values()
    data_qubit_initial_states = {qubit: State.Plus for qubit in data_qubits}
    final_measurements = [Pauli(qubit, PauliLetter('X')) for qubit in data_qubits]
    logical_observables = [code.logical_qubits[observable_index].x]#, code.logical_qubits[1].x]
    stim_circuit: stim.Circuit = compiler.compile_to_stim(code,layers=layers,initial_states= data_qubit_initial_states, final_measurements=final_measurements,logical_observables=logical_observables)
    return(stim_circuit)

def get_Z_stim_circuit(code,observable_index,layers,p):
    compiler = AncillaPerCheckCompiler(noise_model=CircuitLevelNoise(p,p,p,p,p), syndrome_extractor=CxCyCzExtractor())
    data_qubits = code.data_qubits.values()
    data_qubit_initial_states = {qubit: State.Zero for qubit in data_qubits}
    final_measurements = [Pauli(qubit, PauliLetter('Z')) for qubit in data_qubits]
    logical_observables = [code.logical_qubits[observable_index].z]
    stim_circuit: stim.Circuit = compiler.compile_to_stim(code,layers=layers,initial_states= data_qubit_initial_states, final_measurements=final_measurements,logical_observables=logical_observables)
    return(stim_circuit)

In [3]:
circuit = get_X_stim_circuit(HoneycombCode(5), 0,10,0.1)
circuit2 = get_X_stim_circuit(GaugeHoneycombCode(5),0, 5, 0.1)

|████████████████████████████████████████✗︎ (!) 243/213 [114%] in 0.1s (1624.95/s) 
|████████████████████████████████████████✗︎ (!) 483/423 [114%] in 0.3s (1650.53/s)                                       


In [4]:

def dem_histogram(dem):
    weight_dict = dict()
    total_errors = 0
    for dem_object in dem:
        if dem_object.type == "error":
            detector_ids = set()
            observable_ids = set()
            if len(dem_object.targets_copy()) in weight_dict:
                weight_dict[len(dem_object.targets_copy())] += 1
            else:
                weight_dict[len(dem_object.targets_copy())] = 1
            total_errors += 1
            #for target in dem_object.targets_copy():
            #    print('+1')

    print(weight_dict, total_errors)

dem_histogram(circuit.detector_error_model(decompose_errors=True, approximate_disjoint_errors=True))
dem_histogram(circuit2.detector_error_model(decompose_errors=True, approximate_disjoint_errors=True))


{2: 562, 3: 114, 5: 714, 4: 116, 6: 206, 7: 8, 1: 16} 1736
{2: 1240, 3: 180, 4: 284, 5: 1726, 6: 302, 1: 8, 7: 4} 3744


In [10]:
#gauge_hcc_X1_samples = calculate_threshold('GaugeHoneycombCodeX',[4], 1,2,np.linspace(0.001,0.003, 5))
hcc_X1_samples = calculate_threshold('HoneycombCodeX',[12], 1,4,np.linspace(0.001,0.003, 5))

# and save datagauge_hcc_X1_samples = calculate_threshold('GaugeHoneycombCodeX',[4,8,12], 1,2,np.linspace(0.001,0.003, 5))


|████████████████████████████████████████✗︎ (!) 1155/1011 [114%] in 8.4s (138.06/s)                                      ▅▇▇ 456/1011 [45%] in 4s (106.5/s, eta: 5s) 505/1011 [50%] in 5s (110.0/s, eta: 5s)  555/1011 [55%] in 5s (112.2/s, eta: 4s)  ▂▂▄ 595/1011 [59%] in 5s (113.6/s, eta: 4s) ▄▆█ 656/1011 [65%] in 6s (116.5/s, eta: 3s) ▄▂▂ 751/1011 [74%] in 6s (119.5/s, eta: 2s) in 7s (121.4/s, eta: 2s) (129.0/s, eta: 1s) (133.2/s, eta: 0s)                         in 8s (135.9/s, eta: -)  1104/1011 [109%] in 8s (136.9/s, eta: -) 
|████████████████████████████████████████✗︎ (!) 1155/1011 [114%] in 6.8s (169.65/s)                                      ▃▁ 425/1011 [42%] in 4s (115.0/s, eta: 6s)  ▁▃▅ 528/1011 [52%] in 4s (128.1/s, eta: 4s) █▆▄ 664/1011 [66%] in 5s (140.2/s, eta: 3s) (150.2/s, eta: 1s) 
|████████████████████████████████████████✗︎ (!) 1155/1011 [114%] in 5.7s (201.05/s)                                       eta: 6s)  █▆▄ 383/1011 [38%] in 3s (140.1/s, eta: 5s) ▆▄▂ 435/1011 [43%]

Starting workers...
5 cases left:
    beliefmatching processes=1  ~core_mins_left=None    (initializing...) {code=HoneycombCodeX,distance=12,layers=48,p=0.001}
    beliefmatching processes=1  ~core_mins_left=None    (initializing...) {code=HoneycombCodeX,distance=12,layers=48,p=0.0015}
    beliefmatching processes=1  ~core_mins_left=None    (initializing...) {code=HoneycombCodeX,distance=12,layers=48,p=0.002}
    beliefmatching processes=1  ~core_mins_left=None    (initializing...) {code=HoneycombCodeX,distance=12,layers=48,p=0.0025}
    beliefmatching processes=1  ~core_mins_left=None    (initializing...) {code=HoneycombCodeX,distance=12,layers=48,p=0.003}
5 cases left:
    beliefmatching processes=1  ~core_mins_left=None    (initializing...) {code=HoneycombCodeX,distance=12,layers=48,p=0.001}
    beliefmatching processes=1  ~core_mins_left=None    (initializing...) {code=HoneycombCodeX,distance=12,layers=48,p=0.0015}
    beliefmatching processes=1  ~core_mins_left=None    (initializi

KeyboardInterrupt: 